In [8]:
print()
print("******************************")
print("* Ensemble (RoBERTa + TFIDF) *")
print("******************************")


******************************
* Ensemble (RoBERTa + TFIDF) *
******************************


In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import csv
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
train=pd.read_csv(f'data/roberta/roberta.train.csv')
new_train=pd.read_csv(f'data/tfidf/tfidf.train.csv')
test=pd.read_csv(f'data/roberta/roberta.test.csv')
new_test=pd.read_csv(f'data/tfidf/tfidf.test.csv')
eval_set1=pd.read_csv(f'data/roberta/roberta.eval.anon.csv')
eval_set2=pd.read_csv(f'data/tfidf/tfidf.eval.anon.csv')

In [3]:
x_train=train.iloc[:,:-1]
y_train=train.iloc[:,-1]
x_test=test.iloc[:,:-1]
y_test=test.iloc[:,-1]
new_x_train=new_train.iloc[:,:-1]
new_y_train=new_train.iloc[:,-1]
new_x_test=new_test.iloc[:,:-1]
new_y_test=new_test.iloc[:,-1]
eval_x_train1=eval_set1.iloc[:,:-1]
eval_x_train2=eval_set2.iloc[:,:-1]

In [4]:
for i in range(x_train.shape[1]):
    col = x_train.columns[i]
    x_train.rename(columns = {col:'roberta_'+col}, inplace = True)
    
for i in range(new_x_train.shape[1]):
    col = new_x_train.columns[i]
    new_x_train.rename(columns = {col:'tfidf_'+col}, inplace = True)
    
col = new_x_train[new_x_train.columns]
x_train = x_train.join(col)

for i in range(x_test.shape[1]):
    col = x_test.columns[i]
    x_test.rename(columns = {col:'roberta_'+col}, inplace = True)
    
for i in range(new_x_test.shape[1]):
    col = new_x_test.columns[i]
    new_x_test.rename(columns = {col:'tfidf_'+col}, inplace = True)
    
col = new_x_test[new_x_test.columns]
x_test = x_test.join(col)

for i in range(eval_x_train1.shape[1]):
    col = eval_x_train1.columns[i]
    eval_x_train1.rename(columns = {col:'roberta_'+col}, inplace = True)
    
for i in range(eval_x_train2.shape[1]):
    col = eval_x_train2.columns[i]
    eval_x_train2.rename(columns = {col:'tfidf_'+col}, inplace = True)
    
col = eval_x_train2[eval_x_train2.columns]
eval_x_train1 = eval_x_train1.join(col)

In [5]:
dt_classifier = DecisionTreeClassifier(criterion='gini', max_depth=None, random_state=42)
lr_classifier = LogisticRegression(max_iter=1000,random_state=42)
svm_classifier = SVC(kernel='linear',random_state=42)

voting_classifier = VotingClassifier(estimators=[('dt', dt_classifier), 
                                                 ('lr', lr_classifier), 
                                                 ('svm', svm_classifier)], 
                                     voting='hard')

voting_classifier.fit(x_train, y_train)
y_pred = voting_classifier.predict(x_train)
accuracy = accuracy_score(y_train, y_pred)
print("Train set accuracy using voting classifier:",accuracy)
y_pred = voting_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy using voting classifier:",accuracy)

Train set accuracy using voting classifier: 0.9394751693002258
Test set accuracy using voting classifier: 0.8169848584595129


In [6]:
y_pred = voting_classifier.predict(eval_x_train1)

In [7]:
result=pd.DataFrame(y_pred,columns=['label'],dtype=int)
result.index.name='example_id'
result.to_csv('ensemble_results.csv')